# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
# !pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


In [6]:
df.shape

(7, 2)

## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [8]:
#Replace None by your own value and justify

llm = ChatOpenAI(temperature=0.7)

In [9]:
prompt = ChatPromptTemplate.from_template(
    # Query with a variable describing any product
    "Write a compelling, customer-friendly product description for a {product}. "
    "Include: 3 key features, 2 benefits, and a short tagline. "
    "Keep it under 120 words."
)

In [10]:
# Modern "chain" (LCEL): prompt feeds into llm
chain = prompt | llm

In [11]:
product = "wireless noise-cancelling headphones"
result = chain.invoke({"product": product})

In [12]:
# result is an AIMessage; print content:
print(result.content)

Immerse yourself in clear, crisp sound with our wireless noise-cancelling headphones. Featuring advanced noise-cancellation technology, these headphones block out unwanted background noise for a truly immersive listening experience. Enjoy the convenience of wireless connectivity, allowing you to move freely without being tethered to your device. The built-in microphone ensures crystal-clear calls, while the long-lasting battery keeps the music playing all day long. Say goodbye to distractions and hello to premium sound quality with our wireless noise-cancelling headphones. Elevate your listening experience today.

Key Features:
1. Advanced noise-cancellation technology
2. Wireless connectivity
3. Built-in microphone

Benefits:
1. Immersive listening experience
2. Crystal-clear calls

Tagline: "Block out distractions, elevate your sound."


## SimpleSequentialChain

In [13]:
from langchain_core.output_parsers import StrOutputParser

In [14]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Generate a creative and detailed product description for a {product}."
)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Rewrite the following product description to be more persuasive and "
    "marketing-focused. Add emotional appeal and a strong call to action.\n\n"
    "{text}"
)

In [16]:
simple_sequential_chain = (
    first_prompt
    | llm
    | StrOutputParser()
    | second_prompt
    | llm
    | StrOutputParser()
)

In [17]:
product = "smart fitness watch"
result = simple_sequential_chain.invoke({"product": product})

print("PRODUCT:", product)
print(result)

PRODUCT: smart fitness watch
Transform your health and wellness journey with the all-new Smart Fitness Watch. This cutting-edge device is more than just a watch - it's your ultimate companion, helping you achieve your fitness goals and live your best life.

Track your workouts with precision using the heart rate monitor, step tracker, and calorie counter. Get real-time feedback to push yourself further and see results faster than ever before. With features like sleep tracking and guided breathing exercises, you can ensure you're taking care of both your body and mind.

But that's not all - stay connected and on top of your game with notifications for calls, texts, and emails right on your wrist. The Smart Fitness Watch is your one-stop solution for staying on top of your fitness goals and never missing a beat.

Don't settle for mediocrity - upgrade to the Smart Fitness Watch and elevate your health and wellness routine to new heights. Order now and take the first step towards a healthi

In [18]:
product = "ergonomic office chair"
result = simple_sequential_chain.invoke({"product": product})

print("PRODUCT:", product)
print(result)

PRODUCT: ergonomic office chair
Say goodbye to daily office discomfort with the revolutionary ErgoFlex Ergonomic Office Chair. This chair is not just a piece of furniture, it's a game-changer for your workspace.

Imagine sitting down in a chair that perfectly molds to the shape of your body, providing unmatched support and comfort throughout the day. The ErgoFlex features an adjustable seat cushion and height-adjustable armrests to ensure you stay pain-free and focused, no matter how long your workday may be.

What truly sets the ErgoFlex apart is its innovative lumbar support system. Say goodbye to pesky back pain - the ErgoFlex's customizable lumbar cushion will keep your lower back in check, promoting better posture and increased productivity.

Crafted with high-quality, breathable mesh fabric, the ErgoFlex keeps you cool and comfortable, even during your most intense work sessions. The chair's smooth swivel and easy-rolling casters make moving around your office effortless, so you 

**Repeat the above twice for different products**

## SequentialChain

In [19]:
from langchain_core.runnables import RunnablePassthrough

In [20]:
llm = ChatOpenAI(temperature=0.9)
parser = StrOutputParser()


# 1) Detect language
lang_prompt = ChatPromptTemplate.from_template(
    "Identify the language of the following customer review. "
    "Reply with only the language name.\n\nReview:\n{review}"
)

In [21]:
# 2) Translate to English
translate_prompt = ChatPromptTemplate.from_template(
    "Translate the following customer review into English. Preserve tone and meaning.\n\n{review}"
)

In [22]:
# 3) Summarize
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following customer review in 3 bullet points. "
    "Include sentiment (positive/neutral/negative) in the first bullet.\n\n{english_review}"
)

In [23]:
# 4) Follow-up message using multiple prior outputs
followup_prompt = ChatPromptTemplate.from_template(
    "You are customer support. Write a short, polite follow-up message (60–90 words) "
    "responding to the customer. Use the summary AND details from the English review.\n\n"
    "Detected language: {language}\n\n"
    "English review:\n{english_review}\n\n"
    "Summary:\n{summary}\n\n"
    "Follow-up message:"
)

In [24]:
# Build a SequentialChain-like pipeline that carries a dict state forward
overall_chain = (
    RunnablePassthrough()  # passes the input dict along
    .assign(
        language=lang_prompt | llm | parser
    )
    .assign(
        english_review=translate_prompt | llm | parser
    )
    .assign(
        summary=summary_prompt | llm | parser
    )
    .assign(
        followup_message=followup_prompt | llm | parser
    )
)

In [25]:
review1 = df.Review[2]
out1 = overall_chain.invoke({"review": review1})

print("=== REVIEW 1 ===")
print("Language:", out1["language"])
print("\nEnglish review:\n", out1["english_review"])
print("\nSummary:\n", out1["summary"])
print("\nFollow-up:\n", out1["followup_message"])

=== REVIEW 1 ===
Language: English

English review:
 This mattress had a small hole at the top (took forever to find it), and the patches provided did not work, maybe because it's the fabric-like top of the mattress and the patch won't stick. Maybe I was unlucky with a defective mattress, but where is the quality assurance for this company? That should not happen. Emphasis on flat. Because that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.

Summary:
 - Negative sentiment
- Small hole at the top of the mattress, patches provided did not work
- Mattress was flat and company required it to be shipped back for a refund, overall horrible experience

Follow-up:
 Dear Customer,

We are truly sorry to hear about your experience with our mattr

In [26]:
review2 = df.Review[5]  # pick another index that exists in your dataset
out2 = overall_chain.invoke({"review": review2})

print("\n\n=== REVIEW 2 ===")
print("Language:", out2["language"])
print("\nEnglish review:\n", out2["english_review"])
print("\nSummary:\n", out2["summary"])
print("\nFollow-up:\n", out2["followup_message"])



=== REVIEW 2 ===
Language: French

English review:
 I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old lot or counterfeit!?

Summary:
 - Negative: The taste is mediocre
- Negative: The foam doesn't hold well
- Negative: Suspected old lot or counterfeit product

Follow-up:
 Dear valued customer,

Thank you for sharing your feedback with us. We apologize that the taste of our product did not meet your expectations and that you experienced issues with the foam. We take quality control very seriously and would like to investigate this further. Please provide us with more details such as the lot number and purchase location so we can address your concerns promptly. Your satisfaction is our top priority and we appreciate your patience as we work to resolve this issue.

Best regards,
Customer Support Team


**Repeat the above twice for different products or reviews**

## Router Chain

In [27]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [29]:
import json

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableBranch

In [30]:
llm = ChatOpenAI(temperature=0)
to_str = StrOutputParser()

In [31]:
# Build destination expert chains: each takes {"input": "..."} and returns a string
destination_chains = {}
for p in prompt_infos:
    prompt = ChatPromptTemplate.from_template(p["prompt_template"])
    destination_chains[p["name"]] = prompt | llm | to_str

default_chain = ChatPromptTemplate.from_template("{input}") | llm | to_str

In [32]:
# 2) Router prompt (returns JSON)

destinations_str = "\n".join([f'{p["name"]}: {p["description"]}' for p in prompt_infos])

router_template = """Given a raw text input, select the best expert prompt.

Return ONLY valid JSON (no markdown fences) in this format:
{{
  "destination": "physics|math|history|computer science|biology|DEFAULT",
  "next_inputs": "possibly improved version of the input"
}}

CANDIDATE PROMPTS:
{destinations}

INPUT:
{input}

JSON:"""

router_prompt = PromptTemplate.from_template(router_template)

router_chain = router_prompt.partial(destinations=destinations_str) | llm | to_str

In [33]:
def parse_router_json(text: str) -> dict:
    """
    Robust-ish JSON parse:
    - strips whitespace
    - tries to locate the first {...} block if the model adds extra text
    """
    s = text.strip()
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        # fallback: extract the first JSON object
        start = s.find("{")
        end = s.rfind("}")
        if start != -1 and end != -1 and end > start:
            return json.loads(s[start : end + 1])
        raise

In [34]:
def route(inputs: dict) -> dict:
    """
    inputs: {"input": "..."}
    returns: {"destination": "...", "input": "..."} where input is next_inputs
    """
    router_out = router_chain.invoke({"input": inputs["input"]})
    decision = parse_router_json(router_out)

    dest = (decision.get("destination") or "DEFAULT").strip()
    next_in = (decision.get("next_inputs") or inputs["input"]).strip()

    return {"destination": dest, "input": next_in}

router = RunnableLambda(route)

In [35]:
# 3) Branch by destination

def is_dest(name: str):
    return lambda x: x.get("destination") == name

# Each branch expects dict with {"input": "..."} so we drop "destination" before calling the expert
def call_expert(expert_chain):
    return RunnableLambda(lambda x: {"input": x["input"]}) | expert_chain

final_chain = router | RunnableBranch(
    (is_dest("physics"), call_expert(destination_chains["physics"])),
    (is_dest("math"), call_expert(destination_chains["math"])),
    (is_dest("history"), call_expert(destination_chains["history"])),
    (is_dest("computer science"), call_expert(destination_chains["computer science"])),
    (is_dest("biology"), call_expert(destination_chains["biology"])),
    call_expert(default_chain),  # DEFAULT fallback
)

In [36]:
# 4) Run the same examples

print(final_chain.invoke({"input": "What is black body radiation?"}))
print()
print(final_chain.invoke({"input": "what is 2 + 2"}))
print()
print(final_chain.invoke({"input": "Why does every cell in our body contain DNA?"}))

Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures.

The sum of 2 and 2 is 4.

DNA is present in every cell of the human body because it serves as the genetic blueprint that contains the instructions for the development, growth, and functioning of all living organisms. DNA carries the genetic information that determines an individual's traits, such as eye color, height, and susceptibility to certain diseases. 

Each cell in the human body contains a complete set of DNA, known as the genome, which is made up of genes that encode specific proteins and re

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [37]:
tests = [
    "Write pseudocode for checking if a string is a palindrome in O(n).",
    "During the French Revolution, what role did the Committee of Public Safety play?",
    "Why do we get fever when we're sick?",
    "If f(x)=x^2 and g(x)=2x+1, what is (f∘g)(3)?",
    "Explain what happens to time near the speed of light.",
]

for t in tests:
    print("Q:", t)
    print(final_chain.invoke({"input": t}))
    print("-" * 60)

Q: Write pseudocode for checking if a string is a palindrome in O(n).
Sure! Here is the pseudocode for checking if a string is a palindrome in O(n) time complexity:

```
function isPalindrome(str):
    n = length(str)
    for i from 0 to n/2:
        if str[i] != str[n-i-1]:
            return false
    return true
```

This pseudocode iterates through the string from both ends towards the middle, comparing characters at each position. If at any point the characters do not match, it returns false indicating that the string is not a palindrome. If the loop completes without finding any mismatches, it returns true indicating that the string is a palindrome. This algorithm has a time complexity of O(n) as it only requires a single pass through the string.
------------------------------------------------------------
Q: During the French Revolution, what role did the Committee of Public Safety play?
The Committee of Public Safety was a powerful executive committee established during the Fre